In [1]:
import os
from pathlib import Path
PARENT_PATH = Path(os.getcwd()).parent.absolute()
import pandas as pd

from capstone.config import CapstoneConfig
from capstone.data_access import DataClass

In [2]:
config = CapstoneConfig()
config.CURRENT_PATH = PARENT_PATH

data = DataClass(config)

In [7]:
xls = pd.ExcelFile(os.path.join(data.data_path, "Regulatory Requirements.xlsx"))
df = pd.read_excel(xls,'Requirements List')

In [8]:
df

,id,Regulation,Title / Chapter / Sub part,Chapter / Section,Article / Paragraph,Text,Date,Product Type,GxP,Regulatory Requirement Type,Full article reference,Mapped to a standard?,Standard mapping?
0,1,Brazil ANVISA GMP MD,Chapter 2 General Quality System Requirements,2.1. General Provisions,2.1.1,General Quality System Requirements […] Genera...,2013-03-28 00:00:00,Medical Device,GMP,Core,Brazil ANVISA GMP MD 2.1.1,Mapped to a standard,Regulatory Standards Management
1,2,Brazil ANVISA GMP MD,Chapter 2 General Quality System Requirements,2.2. Management responsibility,2.2.1,General Quality System Requirements […] Manage...,2013-03-28 00:00:00,Medical Device,GMP,Core,Brazil ANVISA GMP MD 2.2.1,Mapped to a standard,"Quality Planning, \nQuality Management"
2,3,Brazil ANVISA GMP MD,Chapter 2 General Quality System Requirements,2.2. Management responsibility,2.2.2,General Quality System Requirements […] Manage...,2013-03-28 00:00:00,Medical Device,GMP,Core,Brazil ANVISA GMP MD 2.2.2,Mapped to a standard,Quality Management
3,4,Brazil ANVISA GMP MD,Chapter 2 General Quality System Requirements,2.2. Management responsibility,2.2.3,General Quality System Requirements […] Manage...,2013-03-28 00:00:00,Medical Device,GMP,Core,Brazil ANVISA GMP MD 2.2.3,Mapped to a standard,Quality Management
4,5,Brazil ANVISA GMP MD,Chapter 2 General Quality System Requirements,2.2. Management responsibility,2.2.4,General Quality System Requirements […] Manage...,2013-03-28 00:00:00,Medical Device,GMP,Core,Brazil ANVISA GMP MD 2.2.4,Mapped to a standard,"Training, \nSaMD, \nQuality Management"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10227,10237,EU IVDR 2017/746,"CHAPTER VII\nPOST-MARKET SURVEILLANCE, VIGILAN...",Section 1\nPost-market surveillance,78.2,The post-market surveillance system shall be s...,NaN,Medical device,GMP,Non-core,EU IVDR VII Article 78.2,Mapped to a standard,Medical Device Post Market Surveillance
10228,10238,EU IVDR 2017/746,"CHAPTER VII\nPOST-MARKET SURVEILLANCE, VIGILAN...",Section 1\nPost-market surveillance,78.3,Data gathered by the manufacturer's post-marke...,NaN,Medical device,GMP,Non-core,EU IVDR VII Article 78.3,Mapped to a standard,Medical Device Post Market Surveillance
10229,10239,EU IVDR 2017/746,"CHAPTER VII\nPOST-MARKET SURVEILLANCE, VIGILAN...",Section 1\nPost-market surveillance,78.4,"If, in the course of the post-market surveilla...",NaN,Medical device,GMP,Non-core,EU IVDR VII Article 78.4,Mapped to a standard,Medical Device Post Market Surveillance
10230,10240,EU MDR 2017/745,CHAPTER I\nSCOPE AND DEFINITIONS,NaN,Article 2.35\nDefinitions,"‘economic operator’ means a manufacturer, an a...",NaN,Medical device,GMP,Core,EU MDR Article 2 (35),Mapped to a standard,Medical Device Post Market Surveillance
